Классификация тональности коротких текстов


In [ ]:
!pip install pymorphy2

In [ ]:
!pip install eli5

In [ ]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.tokenize import word_tokenize
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import eli5

In [ ]:
neg = pd.read_csv('/content/drive/MyDrive/NLP/negative.csv', encoding="utf8", sep=';', header=None)
pos = pd.read_csv('/content/drive/MyDrive/NLP/positive.csv', encoding="utf8", sep=';', header=None)

## Препроцессинг

In [ ]:
df_merged = pd.concat([neg, pos], ignore_index=True, sort=False)
df_merged

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2
226830,411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0
226831,411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13
226832,411368797447417856,1386912938,bedowabymir,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,989,254,251,0


In [ ]:
df_merged['sentiment'] = df_merged[4]
df_merged

,0,1,2,3,4,5,6,7,8,9,10,11,sentiment
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2,-1
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0,-1
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0,-1
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0,-1
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226829,411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2,1
226830,411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0,1
226831,411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13,1
226832,411368797447417856,1386912938,bedowabymir,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,989,254,251,0,1


In [ ]:
df_merged['post'] = df_merged[3]
df_merged

,0,1,2,3,4,5,6,7,8,9,10,11,sentiment,post
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2,-1,на работе был полный пиддес :| и так каждое за...
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0,-1,"Коллеги сидят рубятся в Urban terror, а я из-з..."
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0,-1,@elina_4post как говорят обещаного три года жд...
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0,-1,"Желаю хорошего полёта и удачной посадки,я буду..."
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0,-1,"Обновил за каким-то лешим surf, теперь не рабо..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226829,411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2,1,"Спала в родительском доме, на своей кровати......"
226830,411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0,1,RT @jebesilofyt: Эх... Мы немного решили сокра...
226831,411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13,1,"Что происходит со мной, когда в эфире #proacti..."
226832,411368797447417856,1386912938,bedowabymir,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,989,254,251,0,1,"""Любимая,я подарю тебе эту звезду..."" Имя како..."


In [ ]:
df_merged['post'].iloc[5556]

'ну, почему? Почему мне предлагают британского плюшевого котенка сейчас, когда у меня уже есть Сонька?! :('

In [ ]:
df_merged['sentiment'].unique()

array([-1,  1])

In [ ]:
df_merged['post_processed'] = df_merged['post'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df_merged['post_processed'] = df_merged['post_processed'].apply(lambda x: x.lower())

In [ ]:
df_merged['post_processed'].iloc[5556]

'ну почему почему мне предлагают британского плюшевого котенка сейчас когда у меня уже есть сонька '

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df_merged['post_processed'] = df_merged['post_processed'].progress_apply(lambda x: word_tokenize(x))

100%|██████████| 226834/226834 [00:43<00:00, 5253.33it/s]


In [ ]:
df_merged['post_processed'].values

array([list(['на', 'работе', 'был', 'полный', 'пиддес', 'и', 'так', 'каждое', 'закрытие', 'месяца', 'я', 'же', 'свихнусь', 'так', 'd']),
       list(['коллеги', 'сидят', 'рубятся', 'в', 'urban', 'terror', 'а', 'я', 'изза', 'долбанной', 'винды', 'не', 'могу']),
       list(['elina_4post', 'как', 'говорят', 'обещаного', 'три', 'года', 'ждут']),
       ...,
       list(['что', 'происходит', 'со', 'мной', 'когда', 'в', 'эфире', 'proactivefm', 'звучит', 'моя', 'любимая', 'песня', 'dctalk', 'music', 'httptco65kgffd5oo']),
       list(['любимаяя', 'подарю', 'тебе', 'эту', 'звезду', 'имя', 'какой', 'звезды', 'переводится', 'подмышка']),
       list(['ma_che_rie', 'посмотри', 'непытайтесьпокинутьомск', 'сегодня', 'в', 'вавилоне', 'в', '1820', 'я', 'там', 'тоже', 'есть'])],
      dtype=object)

## Лемматизация (приводим к лексемам, нормальным формам)

In [ ]:
morph = pymorphy2.MorphAnalyzer()

morph.parse("продали")  

[Parse(word='продали', tag=OpencorporaTag('VERB,perf,tran plur,past,indc'), normal_form='продать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'продали', 884, 4),))]

In [ ]:
df_merged['post_lemmatized'] = df_merged['post_processed'].progress_apply(lambda x: [morph.parse(w)[0].normal_form for w in x])

100%|██████████| 226834/226834 [10:44<00:00, 351.87it/s]


In [ ]:
df_merged['post_lemmatized'].iloc[5556]

['ну',
 'почему',
 'почему',
 'я',
 'предлагать',
 'британский',
 'плюшевый',
 'котёнок',
 'сейчас',
 'когда',
 'у',
 'я',
 'уже',
 'есть',
 'сонька']

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X = vectorizer.fit_transform(df_merged['post_lemmatized'].apply(lambda x: " ".join(x))) # progress apply показывает progress bar


In [ ]:
X.shape

(226834, 282540)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, df_merged['sentiment'], test_size = 0.3, random_state=42
)


logit = LogisticRegression(random_state=21)

In [ ]:
logit.fit(X_train, y_train)  # 

In [ ]:
y_pred = logit.predict_proba(X_test) # вероятность того, что модель предскажет для значений (0 - нег, 1 - позитив)

In [ ]:
roc_auc_score(y_test, y_pred[:, 1])  # метрика площади под кривой

0.8255126510131923

## Теперь с n-граммами

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))  # уни - и биграммы, смотрим на отдельные слова и пары из двух (если 2,2, то только пары слов)

In [ ]:
X = vectorizer.fit_transform(df_merged['post_lemmatized'].apply(lambda x: " ".join(x))) # progress apply показывает progress bar

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, df_merged['sentiment'], test_size = 0.3, random_state=42
)


logit = LogisticRegression(random_state=21)

In [ ]:
logit.fit(X_train, y_train)  

In [ ]:
y_pred = logit.predict_proba(X_test) # вероятность того, что модель предскажет для значений (0 - нег, 1 - позитив)

In [ ]:
roc_auc_score(y_test, y_pred[:, 1])  #  С биграммами лучше работает чуть чуть

0.8367810622972464

In [ ]:
eli5.show_weights(estimator=logit, feature_names=list(vectorizer.get_feature_names()),
                  top=(50,50))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+9.342,dd
+7.204,спасибо
+6.876,ddd
+6.637,ахи
+5.933,ахах
+5.872,приятно
+5.775,ахахи
+4.702,обожать
+4.666,добрый
+4.437,отличный


## Коллокации ещё лучше (обратить внимание - коллокация, обратить отец - n-грамма)

In [ ]:
from gensim.models import Phrases

df_merged['post_lemmatized_collocate'] = df_merged['post_lemmatized'].apply(lambda x: ' '.join(x))  


In [ ]:
sent_stream = [doc.split(" ") for doc in df_merged['post_lemmatized_collocate']]
bigram = Phrases(sent_stream, min_count = 5, threshold=2) 
# min_count - минимальное количество бинграмм, чтобы стало коллокацией (5 раз встретили сочетание) 
# threshold - чем больше значение, тем меньше будет выведено коллокаций
# внутри есть оценка качества коллокаций, если она ниже границы (threshold(по умолчанию 10), то коллокация не создается)

In [ ]:
bigram[df_merged['post_lemmatized'].iloc[8822]]

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['baziann',
 'просто',
 'я',
 'не_знать',
 'с_учёба',
 'проблема_с',
 'мама',
 'очень_сильно',
 'поругаться',
 'изз',
 'такой_фигня']

In [ ]:
df_merged['post_ngrams'] = df_merged['post_lemmatized'].progress_apply(lambda x:bigram[x])

100%|██████████| 226834/226834 [00:14<00:00, 15496.78it/s]


In [ ]:
df_merged['post_ngrams']

0         [на_работа, быть, полный, пиддес, и, так_кажды...
1         [коллега, сидеть, рубиться_в, urban, terror, а...
2         [elina_4post, как, говорить, обещаной, три_год...
3         [желать_хороший, полёт, и, удачный, посадкия, ...
4         [обновить, за_какимтый, леший_surf, теперь, не...
                                ...                        
226829    [спасть, в, родительский_дом, на, свой, кроват...
226830    [rt_jebesilofyt, эх, мы, немного, решить, сокр...
226831    [что_происходить, с, я, когда, в_эфир, proacti...
226832    [любимаять, подарить, ты, этот, звезда, имя, к...
226833    [ma_che_rie, посмотреть, непытайтесьпокинутьом...
Name: post_ngrams, Length: 226834, dtype: object

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))  # тут дефолтное значение 1,1 так как мы уже объединили нграммы
X = vectorizer.fit_transform(df_merged['post_ngrams'].apply(lambda x: ' '.join(x)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, df_merged['sentiment'], test_size=0.3, random_state=42
)

logit = LogisticRegression(random_state=21)

In [ ]:
logit.fit(X_train, y_train)

In [ ]:
y_pred = logit.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_pred[:, 1])

0.8312641814947642

In [ ]:
eli5.show_weights(estimator=logit, feature_names=list(vectorizer.get_feature_names()),
                  top=(50,50))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+8.261,dd
+6.242,ddd
+4.881,ахахи
+4.864,ахи
+4.695,xd
+4.637,ахах
+4.309,спасибо
+4.097,rt_tukvasociopat
+3.965,с_день
+3.887,dddd


## Auto ML

In [ ]:
parameters = {'C':[1, 10, 20, 100]}

In [ ]:
clf = GridSearchCV(logit, parameters)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

{'C': 10}

In [ ]:
!pip install scikit-optimize

In [ ]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer
search_spaces = {
  'C': Real(0.1, 1e+4),
}
n_iter_search = 20
bayes_search = BayesSearchCV(
    logit, 
    search_spaces, 
    n_iter=n_iter_search, 
    cv=5,
    verbose=3
)
bayes_search.fit(X_train, y_train)

In [ ]:
bayes_search.best_params_

OrderedDict([('C', 3668.140543372544)])

In [ ]:
y_pred = bayes_search.predict_proba(X_test)
roc_auc_score(y_test, y_pred[:, 1])

0.8150249758859083

In [ ]:
y_pred = clf.predict_proba(X_test)
roc_auc_score(y_test, y_pred[:, 1])

0.8364565194646665